BACKEND For Bobyard Fullstack eng challenge 😀

<!-- Using (Bobyard's stack) Django Rest framework with PostgreSQL -->


Database setup using SQLite

In [32]:
!python3 -m venv my_notebook_env
!source my_notebook_env/bin/activate 

In [33]:
!pip install flask flask_cors

In [34]:
import sqlite3

# Connect to a database file named 'my_database.db'
# Use ':memory:' for an in-memory database that disappears when the connection is closed
conn = sqlite3.connect('my_database.db')


In [35]:
cursor = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS comments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    author TEXT NOT NULL,
    text TEXT,
    date TEXT,
    likes INTEGER,
    image TEXT
);
"""
cursor.execute(create_table_query)

In [36]:
# insert json into database table
import json
try:
    with open('./comments.json', 'r') as file:
        data = json.load(file)
    print(len(data['comments']), ' comments loaded')

except FileNotFoundError:
    print("Error: The file 'data.json' was not found.")

16  comments loaded


In [37]:
comment_list = data['comments']

rows_to_insert = [
    (c['id'], c['author'], c['text'], c['date'], c['likes'], c['image'])
    for c in comment_list
]
print(len(rows_to_insert), ' rows to insert')

query = "INSERT OR REPLACE INTO comments VALUES (?, ?, ?, ?, ?, ?)"
cursor.executemany(query, rows_to_insert)
conn.commit()

cursor.execute("SELECT * FROM comments")
rows = cursor.fetchall()
print(len(rows),' rows added to the database')

conn.close()

16  rows to insert
26  rows added to the database


Setting up the API

In [38]:
from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "http://localhost:3000"}})

# lists all comments
@app.route('/api/comments', methods=['GET'])
def get_comments():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM comments")
    rows = cursor.fetchall()
    comments = [row for row in rows]
    conn.close()
    return jsonify(comments)

# Delete existing comment given comment's id
@app.route('/api/comments/<int:id>', methods=['DELETE'])
def delete_comment(id):
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    try:
      cursor.execute("DELETE FROM comments WHERE id=?", (id,))
      conn.commit()
      conn.close()
      return jsonify({"status": "success"}), 200
    except Exception as e:
      return jsonify({"error": str(e)}), 500
    finally:
      if conn:
        conn.close()

# add a comment
@app.route('/api/comments', methods=['POST'])
def add_comment():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    try:
      ADMIN_AUTHOR = 'Admin'
      ADMIN_IMAGE = ''
      cur_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

      new_comment = (
          ADMIN_AUTHOR,
          request.json['text'],
          cur_time,
          0,
          ADMIN_IMAGE
          )
      
      print(new_comment)
      query = "INSERT INTO comments (author, text, date, likes, image) VALUES (?, ?, ?, ?, ?)"
      cursor.execute(query, new_comment)
      conn.commit()
      return jsonify({"status": "success"}), 200
    except Exception as e:
      return jsonify({"error": str(e)}), 500
    finally:
      if conn:
        conn.close()


# Edit text of existing comments
@app.route('/api/comments/<int:id>', methods=['PUT'])
def edit_comment(id):
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    try:
        new_text = request.json['text']
        cursor.execute("UPDATE comments SET text=? WHERE id=?", (new_text, id))
        conn.commit()
        conn.close()
        return jsonify({"status": "success"}), 200
    except Exception as e:
      return jsonify({"error": str(e)}), 500
    finally:
      if conn:
        conn.close()


In [ ]:
# running the api 
if __name__ == "__main__":
    app.run(port=8000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
